## Implementacja #1
#### Działa w czasie $ O(n^2) $

### Znajdowanie długości

In [1]:
def lis(arr):
    n = len(arr)
    f_vals = [1] * n
    for i in range(1, n):
        for j in range(i):
            if arr[j] < arr[i] and f_vals[j] + 1 > f_vals[i]:
                f_vals[i] = f_vals[j] + 1
    print(f_vals)
    return max(f_vals)

###### Kilka testów

In [2]:
a = [3, 1, 5, 7, 2, 4, 9, 3, 17, 3]
print(lis(a))

[1, 1, 2, 3, 2, 3, 4, 3, 5, 3]
5


### Wypisywanie podciągu

###### UWAGA
Konieczne jest zmodyfikowanie funkcji, w której poszukujemy najdłuższego podciągu rosnącego, w taki sposób, aby zapamiętywać w drugiej tablicy indeksy, na których znajdują się elementy poprzednie w podciągu do danych. Szczegółowe wyjaśnienie powyżej.

In [3]:
def lis(arr):
    n = len(arr)
    f_vals = [1] * n
    parents = [-1] * n
    for i in range(1, n):
        for j in range(i):
            if arr[j] < arr[i] and f_vals[j] + 1 > f_vals[i]:
                f_vals[i] = f_vals[j] + 1
                parents[i] = j

    return max(range(n), key=arr.__getitem__), f_vals, parents


""" Tylko printowanie """
def print_lis(arr):
    idx, _, parents = lis(arr)
    
    def recur(i):
        if parents[i] >= 0:
            recur(parents[i])
        print(arr[i], end=' ')
    
    recur(idx)
    print()

    
""" Tutaj pobieranie tablicy wartości """
def get_lis(arr):
    idx, _, parents = lis(arr)
    result = []
    
    def recur(i):
        # If an element still has some parent elements (previous in the sequence),
        # look for its predecessors recursively
        if parents[i] >= 0:
            result.append(recur(parents[i]))
        return arr[i]
        
    recur(idx)
    result.append(arr[idx])
        
    return result


""" Tutaj iteracyjne pobieranie tablicy wartości (niepotrzebna tablica rodziców) """
def get_lis_iter(arr):
    _, F, _ = lis(arr)
    n = len(F)
    start = 0
    res = []
    for i in range(1, n):
        if F[i] > F[start]: start = i
            
    j = start
    res.append(arr[j])
    for i in range(start, -1, -1):
        if F[i] == F[j] - 1 and arr[i] < arr[j]:
            res.append(arr[i])
            j = i
            
    res.reverse()
    return res

###### Kilka testów

In [4]:
a = [3, 1, 5, 7, 2, 4, 9, 3, 17, 3]
print(get_lis(a))
print_lis(a)
print(get_lis_iter(a))

[3, 5, 7, 9, 17]
3 5 7 9 17 
[1, 2, 4, 9, 17]


## II Sposób
#### Działa w czasie $ O(n^2) $, jednakże jest ok. $ 12x $ szybszy niż powyższy

### Omówienie

Idea polega na stworzeniu tablicy tymczasowej, w której będzimy przechowywać tablice wartości. Na każdym kolejnym indeksie będą się znajdowały wartości, które są ostatnimi wartościami ciągów rosnących, składających się z $ i + 1 $ elementów, gdzie $ i $- indeks tablicy. Wyjaśnimy to lepiej na poniższym przykładzie:

    # Dla tablicy wyjściowej:
    arr = [3, 1, 5, 7, 2, 4, 9, 3, 17, 3]
    # Inicjalizujemy tablicę temp, zapisując w niej koniec pierwszego podciągu:
    temp = [[arr[0]]]  -->  temp: [[3]]
    # Kolejne kroki:
    1) arr:  [3, 1, 5, 7, 2, 4, 9, 3, 17, 3]
                 ^
       temp: [[3, 1]]  <--  1 <= 3, więc 1 kończy ciąg 1-elementowy
       
    2) arr:  [3, 1, 5, 7, 2, 4, 9, 3, 17, 3]
                    ^
       temp: [[3, 1], [5]]  <--  5 > 1, więc 5 kończy ciąg 2-elementowy
       
    3) arr:  [3, 1, 5, 7, 2, 4, 9, 3, 17, 3]
                       ^
       temp: [[3, 1], [5], [7]]  <--  7 > 5, więc 7 kończy ciąg 3-elementowy
       
    4) arr:  [3, 1, 5, 7, 2, 4, 9, 3, 17, 3]
                          ^
       temp: [[3, 1], [5, 2], [7]]  <--  2 > 1, więc 2 kończy ciąg 2-elementowy    
       
    5) arr:  [3, 1, 5, 7, 2, 4, 9, 3, 17, 3]
                             ^
       temp: [[3, 1], [5, 2], [7, 4]]  <--  4 > 2, więc 4 kończy ciąg 3-elementowy  
       
    5) arr:  [3, 1, 5, 7, 2, 4, 9, 3, 17, 3]
                                ^
       temp: [[3, 1], [5, 2], [7, 4], [9]]  <--  9 > 4, więc 9 kończy ciąg 4-elementowy
       
    6) arr:  [3, 1, 5, 7, 2, 4, 9, 3, 17, 3]
                                   ^
       temp: [[3, 1], [5, 2], [7, 4, 3], [9]]  <--  3 > 2, więc 3 kończy ciąg 3-elementowy
       
    6) arr:  [3, 1, 5, 7, 2, 4, 9, 3, 17, 3]
                                       ^
       temp: [[3, 1], [5, 2], [7, 4, 3], [9], [17]]  <--  17 > 9, więc 17 kończy 
                                                          ciąg 5-elementowy 
    7) arr:  [3, 1, 5, 7, 2, 4, 9, 3, 17, 3]
                                          ^
       temp: [[3, 1], [5, 2], [7, 4, 3, 3], [9], [17]]  <--  3 > 2, więc 3 kończy 
                                                           ciąg 3-elementowy 

### Znajdowanie długości
##### Z wyznaczaniem minimum za każdym razem

In [5]:
def lis(arr):
    if len(arr) < 2: return len(arr)
    
    temp = [[arr[0]]]
    
    for i in range(1, len(arr)):
        j = len(temp) - 1
        while j >= 0 and arr[i] <= min(temp[j]):
            j -= 1
        
        if j == len(temp) - 1:
            temp.append([arr[i]])
        else:
            temp[j + 1].append(arr[i])
            
    print(temp)
            
    return len(temp)

###### Kilka testów

In [6]:
a = [3, 1, 5, 7, 2, 4, 9, 3, 17, 3]
print(lis(a))

[[3, 1], [5, 2], [7, 4, 3, 3], [9], [17]]
5


## III Sposób
#### Działa w czasie $ O(n \cdot log(n)) $ (z użyciem kopca Min Heap i Binary Searcha)
##### (Mimo to, nie jest to najszybsze rozwiązanie, ponieważ IV Sposób przedstawia lepsze)

In [7]:
_left = lambda i: 2 * i + 1
_right = lambda i: 2 * i + 2
_parent = lambda i: (i - 1) // 2
        
    
def insert_to_min_heap(heap: list, val: object):
    # Add a value as the last node of our Complete Binary Tree
    heap.append(val)
    # Fix heap in order to satisfy a min-heap property
    _heapify_up(heap, len(heap) - 1)
    
    
def get_min_in_heap(heap: list) -> object:
    return None if not heap else heap[0]
    
    
def _swap(heap: list, i, j):
    heap[i], heap[j] = heap[j], heap[i]
    
    
def _heapify_up(heap: list, curr_idx: 'heapify begin index', end_idx: 'heapify end index' = 0):
    while curr_idx > end_idx:
        parent_idx = _parent(curr_idx)
        if heap[curr_idx] < heap[parent_idx]:
            _swap(heap, curr_idx, parent_idx)
        curr_idx = parent_idx
            
    
def _heapify_down(heap: list, curr_idx: 'heapify begin index', end_idx: 'heapify end index'):
    # Loop till the current node has a child smaller than itself
    # We assume that when we enter a node which both children are
    # larger than this node, a subtree which a current node is a
    # root of must fulfill a min-heap property
    while True:
        i = _left(curr_idx)
        j = _right(curr_idx)
        k = curr_idx
        
        if i < end_idx:
            if heap[i] < heap[k]:
                k = i
            if j < end_idx and heap[j] < heap[k]:
                k = j
                
        if k == curr_idx: return
        # Swap the current with the smallest child
        heap[curr_idx], heap[k] = heap[k], heap[curr_idx]
        curr_idx = k
        

def binary_search(arr: 'sorted sequence', el: 'searched element', fn) -> int:
    left_idx = 0
    right_idx = len(arr)-1
    
    while left_idx <= right_idx:
        mid_idx = (left_idx + right_idx) // 2
        if el > fn(arr[mid_idx]):
            left_idx = mid_idx + 1
        else:
            right_idx = mid_idx - 1

    return left_idx if left_idx >= 0 else -1


def lis(arr):
    if len(arr) < 2: return len(arr)
    
    temp = [[arr[0]]]
    
    for i in range(1, len(arr)):
        idx = binary_search(temp, arr[i], get_min_in_heap)

        if idx == len(temp):
            temp.append([arr[i]])
        else:
            insert_to_min_heap(temp[idx], arr[i])
            
    return len(temp)

###### Kilka testów

In [8]:
a = [3, 1, 5, 7, 2, 4, 9, 3, 17, 3]
lis(a)

5

## IV Sposób
#### Działa w czasie $ O(n \cdot log(n)) $ (z użyciem Binary Searcha - najszybszy)

### Omówienie (wstępne)

###### Szczegółowe omówienie tutaj: <br>
LINK: https://cp-algorithms.com/sequences/longest_increasing_subsequence.html <br><br>
W poniższym algorytmie wykorzystujemy nieco inne podejście. Zamiast przechowywać długość najdłuższego podciągu, który kończy się wartością $ A[i] $, gdzie $ A $ - tablica wyjściowa, w tym przypadku przechowujemy w pomocniczej tablicy wartości końcowe ciągów. Dokładniej, to w pomocniczej tablicy pod indeksem $ i $ znajdzie się zawsze ostatni wyraz podciągu rosnącego, którego długość wynosi dokładnie $ i $. Z tego powodu pod zerowym indeksem umieszczamy wartość $ -\infty $, ponieważ ciąg o długości $ 0 $ nie może się kończyć żadną wartością, bo jest to ciąg pusty. Dlaczego akurat $ -\infty $? Aby to wyjaśnić, musimy omówić najpierw, w jaki sposób jest aktualizowana tablica pomocnicza. 
###### Zastosowanie tablicy pomocniczej:
Interesuje nas jedynie jeden ciąg, który jest najdłuższym podciagiem rosnącym. Może być wiele podciągów o tej samej długości, ale nas one nie obchodzą w tym zadaniu (tzn. chcemy znaleźć długość najdłuższego podciągu lub sam podciąg o największej długości, nieważne który, jeżeli jest ich kilka). Musimy sobie również zdać sprawę, że w tym rozwiązaniu nie jest możliwe odtworzenie właściwego podciągu, w taki sposób, jak mogliśmy to zrobić wyżej (sposób, jaki wymyśliłem na pozyskanie rozwiązania w przypadku algorytmu o złożoności $ O(n \cdot log(n)) $ został przedstawiony niżej). Jest to efektem tego, że na bieżąco modyfikowana jest tablica tymczasowych wartości końcowych w taki sposób, że dana wartość zostaje zastąpiona nową tylko wtedy, gdy później w tablicy (którą otrzymaliśmy na wejściu, traktowanej jako ciąg, w którym szukamy najdłuższego podciągu) znaleźliśmy wartość od niej mniejszą i równocześnie poprzednia wartość do zamienianej (w tymczasowej tablicy) jest od niej mniejsza oraz następna jest od niej większa. Chodzi o to, że jeżeli uda nam się znaleźć taką wartość, którą da się umieścić między dwoma końcami znalezionych wcześniej podciągów rosnących, to możemy większą wartość końcową zastąpić tą nową wartością (możemy przedłużyć jeden z podciągów, którego koniec mamy zapisany w tymczasowej tablicy). Robimy tak, bo nowo znaleziona wartość utworzy podciąg rosnący o takiej samej długości jak zastępowana większa wartość i jednocześnie wciąż mamy pewność, że wszystkie zapisane dalej w tablicy pomocniczej wartości są od niej większe, więc są końcami DŁUŻSZYCH podciągów rosnących (innymi słowy, ta zamiana wartości nam nic nie psuje, jeżeli chodzi o wyznaczone końce pozostałych podciągów).
###### A dlaczego w ogóle zamieniamy tę wartość?
Powód jest jeden, mniejsza wartość, zapisana jako wartość końcowa podciągu o pewnej długości $ i $, w przypadku, gdy istnieje kilka podciągów o długości $ i $, które mogą kończyć się różnymi wartościami, daje nam większą szansę na znalezienie podciągu dłuższego niż obecnie najdłuższy, ponieważ w momencie znalezienia wartości większej od tej, którą ostatnio zapisaliśmy wewnątrz tablicy pomocniczej, która jest jednocześnie mniejsza od wartości zapisanej pod następnym indeksem (tj. $ i + 1 $), możemy wówczas zamienić wartość pod indeksem $ i + 1 $ na tę nową wartość, dzięki czemu niejako przedłużymy podciąg, którego ostatnim elementem była wartość z pola o indeksie $ i $ (a stała się nią nowa wartość z pola o indeksie $ i + 1 $). W ten sposób zwiększamy szansę na znalezienie kolejnej wartości, która jest większa od końcowej, w celu przedłużenia tego podciągu (w skrócie: jeżeli mamy kilka podciągów rosnących o tej samej długości, lepiej jest trzymać w pamięci koniec tego, który kończy się najmniejszą wartością, ponieważ dla niego najszybciej znajdziemy wartość, która będzie w stanie go przedłużyć).
###### A jak w takim razie przedłużamy najdłuższy pociąg?
Tutaj rozwiązanie jest już proste. Jeżeli znajdziemy wartość, która jest wartością większą od wartości końcowej obecnie najdłuższego podciągu rosnącego, możemy tę wartość dołączyć do tego podciągu.

### Implementacja wstępna
#### (Aby zauważyć sposób działania - złożoność $ O(n^2) $)

In [9]:
def lis(arr):
    if len(arr) < 2: return len(arr)
    
    n = len(arr)
    inf = float('inf')
    last = [inf] * (n + 1)
    last[0] = -inf
    
    for i in range(n):
        for j in range(1, n + 1):
            if last[j - 1] < arr[i] < last[j]:
                last[j] = arr[i]
    
    print(last)
    
    for i in range(n, 0, -1):
        if last[i] < inf:
            return i
    return 0

###### Kilka testów

In [10]:
a = [3, 1, 5, 7, 2, 4, 9, 3, 17, 3]
lis(a)

[-inf, 1, 2, 3, 9, 17, inf, inf, inf, inf, inf]


5

### Omówienie (usprawnienia)

Po analizie tego, w jaki sposób zapisujemy w pomocniczej tablicy wartości, łatwo dojdziemy do wniosku, że tablica końcowych wartości znalezionych podciągów rosnących zawsze zawiera wartości posortowane w sposób rosnący (bo aby ciąg był dłuższy, musi się kończyć większą wartością - logiczne). Z tego porządku wynika fakt, iż nie musimy przeglądać tej tablicy liniowo. W każdym algorytmie, gdzie modyfikacji ma zostać poddana tylko jedna wartość w danym momencie z posortowanej tablicy i mamy pewność, że jest to wartość unikalna (tu wszystkie są unikalne - nie ma powtórek), możemy znaleźć indeks tej wartości, przy pomocy <ins>**wyszukiwania binarnego**</ins>. Dzięki temu czas działania algorytmu schodzi do $ O(n \cdot log(n)) $.
<br><br>
Kolejne usprawnienia dotyczą pamięci. Jak można zauważyć, niepotrzebnie zawsze trzymamy w tablicy $ n $ wartości, gdy tak naprawdę zazwyczaj istnieje podciąg, którego długość jest znacznie mniejsza od długości ciągu wejściowego. Jeżeli zależy nam na pamięci, wystarczy przedłużać tablicę tylko wtedy, gdy natrafimy na wartość, która jest obecnie największa.

In [11]:
def binary_search(arr, val):
    left_idx = 0
    right_idx = len(arr) - 1
    
    while left_idx <= right_idx:
        mid_idx = (left_idx + right_idx) // 2
        if val > arr[mid_idx]:
            left_idx = mid_idx + 1
        else:
            right_idx = mid_idx - 1

    return left_idx  # It will never exceed the left side of an array


def lis(arr):
    if len(arr) < 2: return len(arr)
    
    n = len(arr)
    last = []
    
    for i in range(n):
        j = binary_search(last, arr[i])
        if j == len(last): last.append(arr[i])
        else: last[j] = arr[i]
    
    print(last)
    
    return len(last)

###### Kilka testów

In [12]:
a = [3, 1, 5, 7, 2, 4, 9, 3, 17, 3]
lis(a)

[1, 2, 3, 9, 17]


5

### Odtwarzanie rozwiązania

Ponieważ tablica, w której przechowujemy końce najdłuższych podciągów, nie zawiera w żaden sposób informacji o ciągu, którego szukamy. Konieczne jest więc utworzenie tablicy rodziców elementów, które umieszczamy w tymczasowej tablicy. Nie jest to jednak takie proste ponieważ tablica tymczasowa nie przechowuje indeksów elementów, więc podczas jej modyfikacji tak naprawdę nie wiemy, jaki ciąg przedłużamy, ale wiemy tylko to, że poprzedni element występuje wcześniej w tablicy wyjściowej. Musimy więc również utworzyć jeszcze jedną tablicę tymczasową, w której równolegle z dodawanymi wartościami do tymczasowego podciągu, będziemy zapisywać odpowiadające im indesky.

In [13]:
def binary_search(arr, val):
    left_idx = 0
    right_idx = len(arr) - 1
    
    while left_idx <= right_idx:
        mid_idx = (left_idx + right_idx) // 2
        if val > arr[mid_idx]:
            left_idx = mid_idx + 1
        else:
            right_idx = mid_idx - 1

    return left_idx  # It will never exceed the left side of an array


def lis(arr):
    if len(arr) < 2: return len(arr)
    
    n = len(arr)
    last = []
    ind  = []
    parents = [-1] * n
    
    for i in range(n):
        j = binary_search(last, arr[i])
        if j == len(last): 
            if j > 0: parents[i] = ind[j - 1]
            ind.append(i)
            last.append(arr[i])
        else: 
            ind[j] = i
            last[j] = arr[i]
            if j > 0: parents[i] = ind[j - 1]
            
    # Get result
    result = [-1] * len(last)
    j = ind[-1]
    for i in range(len(last) - 1, -1, -1):
        result[i] = arr[j]
        j = parents[j]
    
    return result

###### Kilka testów

In [14]:
a = [3, 1, 5, 7, 2, 4, 9, 3, 17, 3]
lis(a)

[1, 2, 4, 9, 17]